In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder
         .master("spark://spark-master:7077")
         .appName("read-minio-defaults")
         .getOrCreate())

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /opt/spark/ivy/cache
The jars for the packages stored in: /opt/spark/ivy/jars
org.apache.spark#spark-hadoop-cloud_2.13 added as a dependency
io.delta#delta-spark_2.13 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-4.0_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-46261f0c-c63c-41b8-a0c1-06fed1fed0b9;1.0
	confs: [default]
	found org.apache.spark#spark-hadoop-cloud_2.13;4.0.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.hadoop#hadoop-aws;3.4.1 in central
	found org.wildfly.openssl#wildfly-openssl;2.2.5.Final in central
	found s

In [3]:
df = spark.read.option("header","true").csv("s3a://bronze/flights/")
df.show(5, truncate=False)

26/01/11 14:14:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
                                                                                

+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
+--------+-----+--------+------+-----------+
only showing top 5 rows


In [4]:
from pyspark.sql.functions import current_date
df_silver = df.withColumn("ingestion_date", current_date())

In [5]:
silver_out = "s3a://silver/flights"

In [6]:
(df_silver.write
    .mode("overwrite")
    .parquet(silver_out))

In [7]:
spark.stop()